# Design and Develop the Prompt Generation System
- Users can input a description of their objective or task and specify a few scenarios along with their expected outputs. 
- Write or adopt sophisticated algorithms, you generate multiple prompt options based on the provided information. 
- This automated prompt generation process saves time and provides a diverse <br/>
range of alternatives to consider. But add an evaluation metrics that check <br/> 
whether the generated prompt candidate aligns with the input description.


### Import Packages

In [11]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../utility')))

In [12]:
from dotenv import load_dotenv,find_dotenv

from rag import create_retriever
from langchain_utility import data_loader, create_chain_rag

### Load OPENAI KEY from .env

In [13]:
# Load OpenAI API key from .env file
load_dotenv(find_dotenv())

True

### Load the dataset and create chunks

In [14]:
chunks = data_loader("../prompts/challenge_doc.txt")

Created a chunk of size 1757, which is longer than the specified 500
Created a chunk of size 1320, which is longer than the specified 500
Created a chunk of size 859, which is longer than the specified 500
Created a chunk of size 517, which is longer than the specified 500
Created a chunk of size 1345, which is longer than the specified 500
Created a chunk of size 1988, which is longer than the specified 500
Created a chunk of size 851, which is longer than the specified 500
Created a chunk of size 1221, which is longer than the specified 500
Created a chunk of size 852, which is longer than the specified 500
2024-01-18 22:30:56,411:logger:data loaded to vector database successfullu


### Create vector database and retriever

In [15]:
retriever = create_retriever(chunks)

embedded weaviate is already listening on port 8079


{"level":"info","msg":"Created shard langchain_3befa79eada749aea183be1ebacf70bd_WMM4etojJlF5 in 695.625µs","time":"2024-01-18T22:30:56+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-18T22:30:56+03:00","took":82665}
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings

In [16]:
retriever

VectorStoreRetriever(tags=['Weaviate', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.weaviate.Weaviate object at 0x7f65e94cf170>)

In [17]:
rag_chain = create_chain_rag(retriever)

2024-01-18 22:30:58,948:logger:data loaded to vector database successfully


In [18]:
rag_chain

{
  context: VectorStoreRetriever(tags=['Weaviate', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.weaviate.Weaviate object at 0x7f65e94cf170>),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="\n        You are a prompt generator which generate a list of prompts for the user\n        question.  Use the following pieces of retrieved context to answer the question. \n        If you don't know the answer, just say that you don't know. \n        Use two sentences maximum and keep the answer concise. return a list of 5 prompts. \n        do not give the generated questions number like 1. 2.\n        separate the questions based on new line.\n        Question: {question} \n        Context: {context} \n        Answer:\n        "))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f65e

In [23]:
generated_prompts = rag_chain.invoke("questions about the challenge documnet")

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/chat_models/openai.py:458: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed

In [24]:
questions = generated_prompts.split('\n')

for index, question in enumerate(questions, 1):
    print(f"{question.strip()}")

1. What are the deliverables for the challenge?
2. When is the interim submission deadline?
3. Where should the code for the challenge be stored?
4. What are the key performance indicators for this week's challenge?
5. What tasks should be included in the final submission?
